In [13]:
import pandas as pd
from sqlalchemy import create_engine, text

# Load the Excel file
df = pd.read_excel('D:\\Budget.xlsx', sheet_name='Sheet1')

# Create a connection to the database
server = 'USER'
database = 'GPTourism'
driver = 'ODBC Driver 17 for SQL Server'
connection_string = f'mssql+pyodbc://@{server}/{database}?driver={driver}'

engine = create_engine(connection_string)

# Step 1: Retrieve existing ResourceId values from the Resource table to avoid duplicates
with engine.connect() as connection:
    existing_resource_ids = pd.read_sql('SELECT BudgetId FROM Budget', connection)['BudgetId'].tolist()

# Step 2: Filter the DataFrame to exclude rows with existing ResourceId values
df_filtered = df[~df['BudgetId'].isin(existing_resource_ids)]

# Ensure valid AdminId values by querying existing AdminId values from the Admin table
with engine.connect() as connection:
    existing_admin_ids = pd.read_sql('SELECT AdminId FROM Admin', connection)['AdminId'].tolist()

# Filter the DataFrame to include only rows with valid AdminId values
df_filtered = df_filtered[df_filtered['AdminId'].isin(existing_admin_ids)]

# Step 3: Insert the filtered data into the Resource table
with engine.connect() as connection:
    # Enable IDENTITY_INSERT
    connection.execute(text('SET IDENTITY_INSERT Budget ON'))
    
    # Write the filtered data to the Resource table
    df_filtered.to_sql('Budget', con=connection, if_exists='append', index=False)
    
    # Disable IDENTITY_INSERT
    connection.execute(text('SET IDENTITY_INSERT Budget OFF'))

print("The data has been saved to the 'Budget' table.")

The data has been saved to the 'Budget' table.
